## Simple ReAct Agent for Calculator
The following code builds a Calculator.
Tools are defined for each calculation function.
Prompts provide a natural language question.
A ReAct agent runs stepwise to generate the right outcome by using the right tools. 

In [13]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool

In [11]:
def add(a: float, b: float) -> float:
  """Add two numbers and returns the result"""
  return a + b
add_tool = FunctionTool.from_defaults(fn=add)

def multiply(a: float, b: float) -> float:
  """Multiply two numbers and returns the result"""
  return a * b
multiply_tool = FunctionTool.from_defaults(fn=multiply)

def subtract(a: float, b: float) -> float:
  """Subtract two numbers and returns the result"""
  return a - b
subtract_tool = FunctionTool.from_defaults(fn=subtract)

def divide(a: float, b: float) -> float:
  """Divide two numbers and returns the result. Handles division by zero error"""
  if b == 0:
    raise ZeroDivisionError("Division by zero is not allowed")
  return a / b
divide_tool = FunctionTool.from_defaults(fn=divide)

# Scientific functions
def power(a: float, b: float) -> float:
  """Raise a number to a power and returns the result"""
  return a ** b
power_tool = FunctionTool.from_defaults(fn=power)

def factorial(n: int) -> int:
  """Calculate the factorial of a non-negative integer and returns the result"""
  if n < 0:
    raise ValueError("Factorial is not defined for negative numbers")
  if n == 0:
    return 1
  else:
    return n * factorial(n-1)
factorial_tool = FunctionTool.from_defaults(fn=factorial)

def sine(x: float) -> float:
  """Calculate the sine of an angle in radians and returns the result. Uses the math library"""
  import math
  return math.sin(x)
sine_tool = FunctionTool.from_defaults(fn=sine)

def cosine(x: float) -> float:
  """Calculate the cosine of an angle in radians and returns the result. Uses the math library"""
  import math
  return math.cos(x)
cosine_tool = FunctionTool.from_defaults(fn=cosine)

def tangent(x: float) -> float:
  """Calculate the tangent of an angle in radians and returns the result. Handles division by zero error"""
  if cosine(x) == 0:
    raise ZeroDivisionError("Tangent is not defined for angles where cosine is zero")
  return sine(x) / cosine(x)
tangent_tool = FunctionTool.from_defaults(fn=tangent)

In [12]:
# list of tools

tools = [multiply_tool, add_tool, subtract_tool, divide_tool, power_tool, factorial_tool, sine_tool, cosine_tool, tangent_tool]

In [4]:
llm = OpenAI(model="gpt-3.5-turbo")
agent = ReActAgent.from_tools(tools, llm=llm, verbose=True)

In [6]:
response = agent.chat("What is 20+(2*4)? Calculate step by step ")

Thought: The user is asking for a step-by-step calculation of an arithmetic expression.
Action: add
Action Input: {}
Observation: Error: add() missing 2 required positional arguments: 'a' and 'b'
Thought: I need to break down the expression into smaller steps to calculate it.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
Thought: I have the result of the multiplication step. Now, I can proceed with the addition.
Action: add
Action Input: {'a': 20, 'b': 8}
Observation: 28
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The result of 20 + (2 * 4) is 28.


In [25]:
response = agent.query("Calculate cos(30) + tan(45)")

Thought: The current language of the user is: English. I need to use tools to help me answer the question.
Action: cosine
Action Input: {'x': 30}
Observation: 0.15425144988758405
Thought: I still need to calculate the tangent of 45 degrees.
Action: tangent
Action Input: {'x': 45}
Observation: 1.6197751905438615
Thought: I have the values for cos(30) and tan(45). Now I can add them together to get the final result.
Action: add
Action Input: {'a': 0.15425144988758405, 'b': 1.6197751905438615}
Observation: 1.7740266404314455
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The result of cos(30) + tan(45) is approximately 1.774.
